In [ ]:
import functions as f
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import scipy.stats as st
from palette import vanguard_palette

In [ ]:
df = pd.read_csv("../data/clean/df_join_clean.csv")
print(f"Shape of final DataFrame: {df.shape}")

### Completion Rates

In [ ]:
# Calculate Completion Rate
completion_status = df.groupby(['client_id', 'visitor_id', 'visit_id'])['process_step'].max() == 4
completion_rate = completion_status.mean().round(2)
print(f"The completion rate for the whole group is: {completion_rate:.2f}")

### Control Group

In [ ]:
df_control = df[df['Variation'] == 'Control']

# Identify completed sessions (where process_step = 4 exists for a given visit_id)
control_completion_status = df_control.groupby(['client_id', 'visit_id'])['process_step'].max() == 4

control_completion_status.head(30)

# Calculate Completion Rate
control_completion_avg = control_completion_status.mean()
print(f"The completion average for the control group is: {control_completion_avg:.2f}")

### Test Group

In [ ]:
df_test = df[df['Variation'] == 'Test']

# Identify completed sessions
test_completion_status = df_test.groupby(['client_id', 'visit_id'])['process_step'].max() == 4



# Calculate Completion Rate
test_completion_avg = test_completion_status.mean().round(2)
print(f"The completion average for the test group is: {test_completion_avg:.2f}")

### Hypothesis Tests

##### Threshold: Vanguard has set this minimum increase in completion rate at 5%. This is the rate at which the projected benefits, in terms of increased user engagement and potential revenue, are estimated to outweigh the costs of the new design.

#### Completion Rate Hypothesis Test

$$H_{0}: completion-rate-test <= completion-rate_control$$
$$H_{1}: completion-rate-test > completion-rate-control$$

In [ ]:
total_session_started = df_test.groupby('visit_id')['session_started'].max().sum()
total_session_completed = df_test.groupby('visit_id')['session_completed'].max().sum()
total_session_completed

In [ ]:
#Set Hypothesis

#The test group completion rate is performing better on average than the control group.

alpha = 0.05

#Proportion Test

# Compute test statistics

x_test = df_test.groupby('visit_id')['session_completed'].max().sum()  # Completed in test group
x_control = df_control.groupby('visit_id')['session_completed'].max().sum() # Completed in control group

n_test = df_test['visit_id'].nunique()   # Unique visits in test group
n_control = df_control['visit_id'].nunique()  # Unique visits in control group

p1_test = x_test / n_test  # Completion rate test group
p1_control = x_control / n_control  # Completion rate control group

# Compute pooled proportion and standard error
p_pooled = (x_test + x_control) / (n_test + n_control)  # Pooled proportion
q_pooled = 1 - p_pooled  # Complement of pooled proportion

SE = np.sqrt(p_pooled * q_pooled * (1/n_test + 1/n_control))

# Compute Z-score
z = (p1_test - p1_control) / SE

# Compute one-tailed p-value (right-tailed test)
p_value = 1 - st.norm.cdf(z)

# Hypothesis test decision
if p_value < alpha:
    print("Reject the null hypothesis: The test group has a significantly higher completion rate than the control group.")
else:
    print("Fail to reject the null hypothesis: There is no statistically significant evidence that the test group performs better than the control group.")

print(f"Z-score: {z:.4f}, p-value: {p_value}")

$$H_{0}: \text{completion\_rate\_test} - \text{completion\_rate\_control} >= 0.05$$
$$H_{1}: \text{completion\_rate\_test} - \text{completion\_rate\_control} < 0.05$$

In [ ]:
#Proportion Test

# Compute test statistics
x_test = df_test.groupby('visit_id')['session_completed'].max().sum()  # Completed in test group
x_control = df_control.groupby('visit_id')['session_completed'].max().sum() # Completed in control group

n_test = df_test['visit_id'].nunique()   # Unique visits in test group
n_control = df_control['visit_id'].nunique()  # Unique visits in control group

p1_test = x_test / n_test  # Completion rate test group
p1_control = x_control / n_control  # Completion rate control group

# Define the expected difference under H0
delta = 0.05  # 5% increase

# Compute standard error
p_pooled = (x_test + x_control) / (n_test + n_control)  # Pooled proportion
q_pooled = 1 - p_pooled

SE = np.sqrt(p_pooled * q_pooled * (1/n_test + 1/n_control))

# Compute Z-score
z = (p1_test - p1_control - delta) / SE  # Subtract delta

# Compute one-tailed p-value (right-tailed test)
p_value = 1 - st.norm.cdf(z)

# Print results
print(f"Z-score: {z:.4f}, p-value: {p_value:.6f}")

# Hypothesis test decision
if p_value < alpha:
    print("Reject the null hypothesis: The completion rate increase is greater than 5%.")
else:
    print("Fail to reject the null hypothesis: The completion rate increase is at most 5%.")

#### Average Client Tenure Hypothesis Test

$$H_{0}: \text{avg\_age\_test} <= \text{avg\_age\_control}$$
$$H_{1}: \text{avg\_age\_test} > \text{avg\_age\_control}$$

In [ ]:
#The average age of clients engaging with the new process is the same as those engaging with the old process.

#Chisquare Test

crosstab_age = pd.crosstab(df['Variation'], df['age_group'])
chi2_stat, chi2_p_value, _, expected_frequency = st.chi2_contingency(crosstab_age)

chi2_stat,chi2_p_value

if chi2_p_value > alpha:
    print(f"Fail to reject the null hypothesis (H0). There is no significant difference in age distribution between the test and control groups (p-value: {chi2_p_value:.4f}).")
else:
    print(f"Reject the null hypothesis (H0). There is a significant difference in age distribution between the test and control groups (p-value: {chi2_p_value:.4f}).")

#### Average Client Tenure Hypothesis Test

$$H_{0}: \text{avg\_tenure\_test} = \text{avg\_tenure\_control}$$
$$H_{1}: \text{avg\_tenure\_test} \_  != \text{avg\_tenure\_control}$$

In [ ]:
#Set Hypothesis

#The average client tenure engaging with the new process is the same as those engaging with the old process.

#H0  avg_tenure_test = avg_tenure_control
#H1: avg_tenure_test != avg_tenure_control


tenure_control_df = df[(df['Variation'] == 'Control')]['clnt_tenure_yr']
tenure_test_df = df[(df['Variation'] == 'Test')]['clnt_tenure_yr']

stat, p_value = st.ttest_ind(tenure_test_df, tenure_control_df, equal_var=False, alternative='two-sided')

if p_value > alpha:
    print(f"We fail to reject the null hypothesis (H0). There is no significant difference in average tenure between the test and control groups (p-value: {p_value}).")
else:
    print(f"We reject the null hypothesis (H0). There is a significant difference in average tenure between the test and control groups (p-value: {p_value}).")